## 1. Data Collection
For simplicity, let's use the yfinance library to fetch stock prices and assume we have a CSV file (macro_data.csv) for macroeconomic indicators:

In [1]:
import yfinance as yf
import pandas as pd

def fetch_stock_data(ticker, start_date, end_date):
    stock_data = yf.download(ticker, start=start_date, end=end_date)
    return stock_data['Close']

def fetch_macro_data():
    return pd.read_csv('macro_data.csv')


## 2. Modeling
This is a complex step, and the actual models can vary widely based on the data and the specific forecasting task. For simplicity, let's assume a basic linear regression model using statsmodels:

In [2]:
'''import statsmodels.api as sm

def forecast_gdp(macro_data, stock_data):
    X = sm.add_constant(stock_data)
    y = macro_data['GDP']
    model = sm.OLS(y, X).fit()
    forecast = model.predict(X)
    return forecast
'''
import statsmodels.api as sm
from statsmodels.tsa.api import VAR

def fit_var_model(data):
    model = VAR(data)
    model_fitted = model.fit(maxlags=15, ic='aic')
    forecast = model_fitted.forecast(data.values[-model_fitted.k_ar:], steps=5)
    return forecast


## 3. Dash app creation

In [4]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd

app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Macroeconomic Forecasting and Financial Market Linkages"),
    dcc.Input(id='ticker-input', value='^GSPC', type='text', placeholder="Enter stock ticker (e.g., ^GSPC for S&P 500)"),
    dcc.DatePickerRange(
        id='date-picker',
        start_date='2010-01-01',
        end_date='2020-01-01',
        display_format='YYYY-MM-DD'
    ),
    html.Button('Forecast', id='submit-button'),
    html.Div(id='output-div')
])

@app.callback(
    Output('output-div', 'children'),
    [Input('submit-button', 'n_clicks')],
    [dash.dependencies.State('ticker-input', 'value'),
     dash.dependencies.State('date-picker', 'start_date'),
     dash.dependencies.State('date-picker', 'end_date')]
)
def update_output(n_clicks, ticker, start_date, end_date):
    if n_clicks is None:
        return "Enter a ticker and date range to forecast macroeconomic indicators."

    stock_data = fetch_stock_data(ticker, start_date, end_date)
    macro_data = fetch_macro_data()

    # Merge stock data with macro data on date
    combined_data = pd.merge(stock_data, macro_data, left_index=True, right_index=True)
    
    forecast = fit_var_model(combined_data)
    gdp_forecast = forecast[:, 0]  # Assuming GDP is the first column
    unemployment_forecast = forecast[:, 1]  # Assuming unemployment is the second column

    return html.Div([
        html.H3(f"Forecasted GDP: {gdp_forecast[-1]:.2f}"),
        html.H3(f"Forecasted Unemployment Rate: {unemployment_forecast[-1]:.2f}%")
    ])

if __name__ == '__main__':
    app.run_server(debug=True)


Address already in use
Port 8050 is in use by another program. Either identify and stop that program, or start the server with a different port.


AttributeError: 'tuple' object has no attribute 'tb_frame'